20240908 整理自  https://weread.qq.com/web/reader/0403218071e8dae00406ed9k977321c02529778d5d2116b

源码库： https://github.com/davecom/ClassicComputerScienceProblemsInPython/tree/master/Chapter8


在四子棋游戏中，两名玩家在 **7 列 6 行** 的垂直棋盘网格中交替落下各自不同颜色的棋子。

棋子从**棋盘网格顶部**往底部下落，直至碰到底部或其他棋子。其实玩家在每个回合中唯一要做的**决策**就是**把棋子落入 7 列中的哪一列**。玩家可能不会把棋子落入全满的列。

只要首先有 4 个同色棋子沿着行、列或对角线紧密相连，中间没有断开，则其玩家就获胜。如果没有玩家能做到这一点，且棋盘网格被完全填满，那么游戏结果就是平局。

### 棋盘游戏的基础组件

In [17]:
# board.py

from __future__ import annotations
from typing import NewType, List
from abc import ABC, abstractmethod

Move = NewType('Move', int)  # Move


class Piece: # 棋子
    @property
    def opposite(self) -> Piece:
        raise NotImplementedError("Should be implemented by subclasses.")


class Board(ABC):
    @property
    @abstractmethod
    def turn(self) -> Piece:  # 该轮到谁走棋了？
        ...

    @abstractmethod
    def move(self, location: Move) -> Board:  # 一步棋
        ...

    @property
    @abstractmethod
    def legal_moves(self) -> List[Move]: # 在当前位置上有哪些符合规则的走法？
        ...

    @property
    @abstractmethod
    def is_win(self) -> bool: # 游戏有人赢了吗？
        ...

    @property
    def is_draw(self) -> bool: # 游戏平局了吗？等效于 (没人赢 + 没有符合规则的棋步可走)
        return (not self.is_win) and (len(self.legal_moves) == 0)

    @abstractmethod
    def evaluate(self, player: Piece) -> float: # 评估当前位置，看看哪位玩家占据了优势。
        ...

## 8.3 四子棋



### 8.3.1 四子棋游戏程序

In [18]:
# connectfour.py

from __future__ import annotations
from typing import List, Optional, Tuple
from enum import Enum
# from board import Piece, Board, Move


class C4Piece(Piece, Enum):
    B = "B"  # 黑色 black
    R = "R"  # 红色 red
    E = " " # stand-in for empty

    @property
    def opposite(self) -> C4Piece:
        if self == C4Piece.B:
            return C4Piece.R
        elif self == C4Piece.R:
            return C4Piece.B
        else:
            return C4Piece.E

    def __str__(self) -> str:
        return self.value

# 指定大小的四子棋棋盘网格中生成 可能赢棋 的所有网格区段（segment）
## 返回 列/行 组成的元组
def generate_segments(num_columns: int, num_rows: int, segment_length: int) -> List[List[Tuple[int, int]]]:
    segments: List[List[Tuple[int, int]]] = []
    # generate the vertical segments
    for c in range(num_columns):
        for r in range(num_rows - segment_length + 1):
            segment: List[Tuple[int, int]] = []
            for t in range(segment_length):
                segment.append((c, r + t))
            segments.append(segment)

    # generate the horizontal segments
    for c in range(num_columns - segment_length + 1):
        for r in range(num_rows):
            segment = []
            for t in range(segment_length):
                segment.append((c + t, r))
            segments.append(segment)

    # generate the bottom left to top right diagonal segments
    for c in range(num_columns - segment_length + 1):
        for r in range(num_rows - segment_length + 1):
            segment = []
            for t in range(segment_length):
                segment.append((c + t, r + t))
            segments.append(segment)

    # generate the top left to bottom right diagonal segments
    for c in range(num_columns - segment_length + 1):
        for r in range(segment_length - 1, num_rows):
            segment = []
            for t in range(segment_length):
                segment.append((c + t, r - t))
            segments.append(segment)
    return segments


class C4Board(Board):
    NUM_ROWS: int = 6
    NUM_COLUMNS: int = 7
    SEGMENT_LENGTH: int = 4
    SEGMENTS: List[List[Tuple[int, int]]] = generate_segments(NUM_COLUMNS, NUM_ROWS, SEGMENT_LENGTH)
# 将四子棋棋盘视为 7 列的组合， 能够让 C4Board 类的其余部分更加容易编写。
    class Column:
        def __init__(self) -> None:
            self._container: List[C4Piece] = []

        @property
        def full(self) -> bool:
            return len(self._container) == C4Board.NUM_ROWS

        def push(self, item: C4Piece) -> None:
            if self.full:
                raise OverflowError("Trying to push piece to full column")
            self._container.append(item)

        def __getitem__(self, index: int) -> C4Piece:
            if index > len(self._container) - 1:
                return C4Piece.E
            return self._container[index]

        def __repr__(self) -> str:
            return repr(self._container)

        def copy(self) -> C4Board.Column:
            temp: C4Board.Column = C4Board.Column()
            temp._container = self._container.copy()
            return temp
## 以下 4 个 方法 与 井字棋 的 对应方法 类似
    def __init__(self, position: Optional[List[C4Board.Column]] = None, turn: C4Piece = C4Piece.B) -> None:
        if position is None:
            self.position: List[C4Board.Column] = [C4Board.Column() for _ in range(C4Board.NUM_COLUMNS)]
        else:
            self.position = position
        self._turn: C4Piece = turn

    @property
    def turn(self) -> Piece:
        return self._turn

    def move(self, location: Move) -> Board:
        temp_position: List[C4Board.Column] = self.position.copy()
        for c in range(C4Board.NUM_COLUMNS):
            temp_position[c] = self.position[c].copy()
        temp_position[location].push(self._turn)
        return C4Board(temp_position, self._turn.opposite)

    @property
    def legal_moves(self) -> List[Move]:
        return [Move(c) for c in range(C4Board.NUM_COLUMNS) if not self.position[c].full]

    # 返回指定区段中 黑色 和 红色棋子的数量
    # Returns the count of black & red pieces in a segment
    def _count_segment(self, segment: List[Tuple[int, int]]) -> Tuple[int, int]:
        black_count: int = 0
        red_count: int = 0
        for column, row in segment:
            if self.position[column][row] == C4Piece.B:
                black_count += 1
            elif self.position[column][row] == C4Piece.R:
                red_count += 1
        return black_count, red_count

    @property
    def is_win(self) -> bool:  # 查看棋盘中的所有区段来确定是否有人赢
        for segment in C4Board.SEGMENTS:
            black_count, red_count = self._count_segment(segment)
            if black_count == 4 or red_count == 4: # 是否有区段包含 4 个同色棋子
                return True
        return False

# 包含两个同色棋子和两个空棋子的区段将被视为得 1 分。
# 包含 3 个同色棋子得分为 100。
# 最后，包含 4 个同色棋子（有人赢棋）的区段得分为 1000000。
# 如果该区段属于对手，则得分为负数。
    ## 评估 辅助函数
    def _evaluate_segment(self, segment: List[Tuple[int, int]], player: Piece) -> float:
        black_count, red_count = self._count_segment(segment)
        if red_count > 0 and black_count > 0:
            return 0 # mixed segments are neutral
        count: int = max(red_count, black_count)
        score: float = 0
        if count == 2:
            score = 1
        elif count == 3:
            score = 100
        elif count == 4:
            score = 1000000
        color: C4Piece = C4Piece.B
        if red_count > black_count:
            color = C4Piece.R
        if color != player:
            return -score
        return score

    def evaluate(self, player: Piece) -> float: # 总分
        total: float = 0
        for segment in C4Board.SEGMENTS:
            total += self._evaluate_segment(segment, player)
        return total

    def __repr__(self) -> str:
        display: str = ""
        for r in reversed(range(C4Board.NUM_ROWS)):
            display += "|"
            for c in range(C4Board.NUM_COLUMNS):
                display += f"{self.position[c][r]}" + "|"
            display += "\n"
        return display

### 8.2.2 极小化极大算法

In [19]:
# minimax.py

from __future__ import annotations
# from board import Piece, Board, Move

### 方法一;
# Find the best possible outcome for original player
def minimax(board: Board, maximizing: bool, original_player: Piece, max_depth: int = 8) -> float:
    # Base case – terminal position or maximum depth reached
    if board.is_win or board.is_draw or max_depth == 0:
        return board.evaluate(original_player)

    # Recursive case - maximize your gains or minimize the opponent's gains
    if maximizing:
        best_eval: float = float("-inf") # arbitrarily low starting point
        for move in board.legal_moves:
            result: float = minimax(board.move(move), False, original_player, max_depth - 1)
            best_eval = max(result, best_eval) # we want the move with the highest evaluation
        return best_eval
    else: # minimizing
        worst_eval: float = float("inf")
        for move in board.legal_moves:
            result = minimax(board.move(move), True, original_player, max_depth - 1)
            worst_eval = min(result, worst_eval) # we want the move with the lowest evaluation
        return worst_eval


def alphabeta(board: Board, maximizing: bool, original_player: Piece, max_depth: int = 8, alpha: float = float("-inf"), beta: float = float("inf")) -> float:
    # Base case – terminal position or maximum depth reached
    if board.is_win or board.is_draw or max_depth == 0: # 递归跳出条件： 达到终局状态 或 最大深度
        return board.evaluate(original_player)

    # Recursive case - maximize your gains or minimize the opponent's gains
    if maximizing:
        for move in board.legal_moves:
            result: float = alphabeta(board.move(move), False, original_player, max_depth - 1, alpha, beta)
            alpha = max(result, alpha)
            if beta <= alpha:
                break
        return alpha
    else:  # minimizing
        for move in board.legal_moves:
            result = alphabeta(board.move(move), True, original_player, max_depth - 1, alpha, beta)
            beta = min(result, beta)
            if beta <= alpha:
                break
        return beta


# 为某棋局中每一步合法的走法循环调用 minimax(),找出评分最高的走法
# Find the best possible move in the current position
# looking up to max_depth ahead
def find_best_move(board: Board, max_depth: int = 3) -> Move:  # 修改： 改成 3
    best_eval: float = float("-inf")
    best_move: Move = Move(-1)
    for move in board.legal_moves:
        result: float = alphabeta(board.move(move), False, board.turn, max_depth)
        if result > best_eval:
            best_eval = result
            best_move = move
    return best_move



### 8.3.2 四子棋 AI

max_depth 设成了 3。


*  这里的四子棋 AI 最多只能看到后 3 步的（评分）棋局。


<font color=blue>输入你要下的列的编号 ！！！</font>

7 列 6 行



In [20]:
0# connectfour_ai.py

# from minimax import find_best_move
# from connectfour import C4Board
# from board import Move, Board

board: Board = C4Board()


def get_player_move() -> Move:
    player_move: Move = Move(-1)
    while player_move not in board.legal_moves:
        play: int = int(input("Enter a legal column (0-6):"))
        player_move = Move(play)
    return player_move


if __name__ == "__main__":
    # main game loop
    while True:
        human_move: Move = get_player_move()
        board = board.move(human_move)
        if board.is_win:
            print("Human wins!")
            break
        elif board.is_draw:
            print("Draw!")
            break
        computer_move: Move = find_best_move(board, 5)
        print(f"Computer move is {computer_move}")
        board = board.move(computer_move)
        print(board)
        if board.is_win:
            print("Computer wins!")
            break
        elif board.is_draw:
            print("Draw!")
            break

Computer move is 2
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|B| |R| | | | |

Computer move is 4
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|B| | | | | | |
|B| |R| |R| | |

Computer move is 0
| | | | | | | |
| | | | | | | |
|R| | | | | | |
|B| | | | | | |
|B| | | | | | |
|B| |R| |R| | |

Computer move is 3
| | | | | | | |
| | | | | | | |
|R| | | | | | |
|B| | | | | | |
|B| | |R| | | |
|B| |R|B|R| | |

Computer move is 3
| | | | | | | |
| | | | | | | |
|R| | | | | | |
|B| | |R| | | |
|B| | |R|B| | |
|B| |R|B|R| | |

Computer move is 4
| | | | | | | |
| | | | | | | |
|R| | |B| | | |
|B| | |R|R| | |
|B| | |R|B| | |
|B| |R|B|R| | |

Computer move is 2
| | | | | | | |
| | | | | | | |
|R| | |B| | | |
|B| |R|R|R| | |
|B| |B|R|B| | |
|B| |R|B|R| | |

Computer move is 5
| | | | | | | |
| | | | | | | |
|R| |B|B| | | |
|B| |R|R|R| | |
|B| |B|R|B| | |
|B| |R|B|R|R| |

Computer move is 3
| | | | | | | |
| | | |R| | | |
|R| |B|B|B| | |
|B| |

通过增加搜索的深度，我们可以提升它的游戏水平，但计算机每走一步的计算时间将呈指数级增长。

<mark>最佳的四子棋开局走法是把棋子放在中间列。</font>

### 8.3.3 用 α-β 剪枝算法优化极小化极大算法

在搜索时能**将不会生成更优结果的棋局排除**，由此来**增加搜索的深度**。


α 表示搜索树当前找到的最优极大化走法的评分，而 β 则表示当前找到的**对手**的最优极小化走法的评分。
* 如果 β $\leq$ α，则不值得对该搜索分支做进一步搜索，因为已经发现的走法比继续沿着该分支搜索得到的走法都要好或相当。这种启发式算法能显著缩小搜索空间。

minimax() 的搜索深度为 5 时，四子棋 AI 每步大约耗时 3 分钟，而相同深度条件下用 alphabeta() 每步大约耗时 30 秒，只需要六分之一的时间！

## 优化技术

### 优化技术：迭代加深（iterative deepening）​

有一种常见的优化技术就是迭代加深（iterative deepening）​。在迭代加深技术中，搜索函数将**先以最大深度 1 运行**，然后以最大深度 2 运行，再以最大深度 3 运行，依次类推。**达到指定时限时，搜索停止**。最后一次完成的搜索深度的结果将会被返回。

### 优化技术：静态搜索（quiescence search）​


在静态搜索技术中，极小化极大搜索树将朝着会让棋局发生巨大变化的路线（如国际象棋中的吃子）行进，而不是朝着相对“平静”的棋局发展。理想情况下，采用这种方案搜索**不会将计算时间浪费在**无聊的棋局上，也就是**那些不会让玩家获得明显优势的棋局。**

极小化极大搜索的最佳优化方案不外乎两种，一种是**在规定的时间内搜索更深的深度**，另一种就是**改进棋局评分函数**。

要在相同时间内搜索更多的棋局，就需要减少在每个棋局上耗费的时间，这可以通过**提高代码效率或采用运行速度更快的硬件**而获得，但也可能会通过后一种改进技术（改进棋局评分函数）而获得。

采用更多的参数或启发式算法来对棋局进行评分可能会耗费更多的时间，但最终能够获得更优质的引擎，即**用更少的搜索深度找到最优走法**。

大多数传统棋盘游戏（如跳棋、国际象棋、四子棋、拼字游戏等）的**搜索空间都比较小**，**基于极小化极大算法的技术可足以应对了**。